<a href="https://colab.research.google.com/github/yshung1971/myfirstproject/blob/main/CandyCrushP2V1.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Candy Crush Game Part 2 (Workable)

Now is Workable 6/5/2022 V0.8
* fix list:
    * [X] Rewrite crushOut() to separate to two functions: checkIsStable() and crushOut()(V1.0)
    * [X] Display the coordinate besides the board.(V1.0)
    * [ ] Now only a basic score sytem is implemented. Plan to design more complex score system. (V2.0)
    * [X] Input validation.(V1.0)




In [232]:

import random

ROW = 10    # (5 ~ 12)
COL = 6     # (5 ~ 12)
ROUNDS = 3
CANDY_TYPES = 5 # (4 ~ 8)

current_score = 0
EMPTY = 0
DEBUG = False


# Debug 
def printd(*args, end = '\n'):
    if DEBUG == True:
        print("DBG:",*args, end = end)  


# Create the board
def createBoard():
    board = [[0 for col in range(COL)] for row in range(ROW)]
    return board


# Display the board
def displayBoard(board, rounds = 0, score = 0, is_score = False):
    num_separator = COL * 4 + 4
    if is_score == True:
        print()
        print(f"ROUND: {ROUNDS - rounds} SCORE: {score}")
        print("="*num_separator)

    # print head
    print("   +","---+"*(COL), sep="")
    for i in range(ROW):
        print(f"{i:2d}","| ", end="")
        for j in range(COL):
            print(board[i][j], end=" | ")
        print()
        print("   +","---+"*(COL), sep="")
    print("     ", end="")
    for j in range(COL):
        print(chr(97+j), end="   ")
    print()
    


# Find all consecutive-squares which is more than 3
def getConsecutiveSquares(board):
    consecutive_squares = []
    _ = []

    # Horizontal
    num = COL
    for row in range(ROW):
        for j in range(COL, 2, -1):
            for col in range(COL - j + 1):
                for i in range(j):
                    _.append([row, col + i])
                consecutive_squares.append(_)
                _ = []
    
    # Vertical
    num = ROW
    for col in range(COL):
        for i in range(ROW, 2, -1):
            for row in range(ROW - i + 1):
                for j in range(i):
                    _.append([row + j, col])
                consecutive_squares.append(_)
                _ = []

    return consecutive_squares



# Score System
def scoreSystem(same_consecutive_squares, board):
    printd(f"scoreSystem(): {same_consecutive_squares}")

    # Use the properties of the set container to remove duplicates of same_consecutive_squares list.
    # The purpose is to calalculate how many candies actually are removed. Every removed candy will
    # get a score.
    # my_tup = ()
    # my_set = set()
    # for outer_lst in same_consecutive_squares:
    #     inner_lsts = []
    #     for inner_lst in outer_lst:
    #         inner_lsts.append(tuple(inner_lst))
    #     print(inner_lsts)
    #     my_tup += tuple(inner_lsts)
    
    # my_set = set(my_tup)
    # print(f"my_tup {my_tup}")
    # print(f"my_set {my_set}")

    return 0



# Check if the board is stable
def checkIsStable(board):
    consecutive_squares = getConsecutiveSquares(board)
    consecutive_square_contents = [] # each_consecutive_square_contents
    same_consecutive_squares = [] # index combinations which value is the same

    # Find consecutive squares which have the same contents
    for _ in consecutive_squares:

        for coord in _:
            row, col = coord
            consecutive_square_contents.append(board[row][col])

        # In python, the elements in set container is unique. That means we can use set() to 
        # container to remove duplicate elements in list, i.e., my_list = [1, 2, 2, 3, 3, 3],
        # set(my_list) will be equal to {1, 2, 3}. (Be careful when using set() in a nested list.)
        if len(set(consecutive_square_contents)) == 1 and consecutive_square_contents[0] != 0:
            printd(f"{consecutive_square_contents} has the same value!")
            same_consecutive_squares.append(_)

        consecutive_square_contents = []
    printd(f"same_consecutive_squares {same_consecutive_squares}")

    if len(same_consecutive_squares) == 0:
        return True, None
    else:
        return False, same_consecutive_squares



# Crush out candies
def crushOut(consecutive_squares, board):
    score = 0
    tmp_score = 0

    is_stable, same_consecutive_squares = checkIsStable(board)
    
    if not is_stable:

        score = scoreSystem(same_consecutive_squares, board)

        # Change these consecutive squares we found to 0 (empty squares)
        for _ in same_consecutive_squares:
            for coord in _:
                row, col = coord
                board[row][col] = 0
        printd(f"after remove: board: {board}")

        # Calculate score
        for row in range(ROW):
            for col in range(COL):
                if board[row][col] == 0:
                    score += 1

        printd(f"score: {score}")

        print()
        print("Found the same candies more than 3 in-a-row!")
        print("Crushing out!")
        displayBoard(board)

        # return new board, is board stable?, score
        return board, False, score
    else:
        return board, True, score
    


# After crush out candies, move down the rest of the candies above them.
def moveDown(board):
    '''
        This function is more complicated. So here is the explanation:
        *
        if there is a 2D list as follows:

                [
                [1, 2, 3], 
                [4, 0, 3],
                [0, 8, 0],
                [0, 2, 0],
                [4, 0, 0]]

        Variable: vertial_index will be equal to = [
                [
                [[0, 0], [1, 0], [2, 0], [3, 0], [4, 0]], 
                [[0, 1], [1, 1], [2, 1], [3, 1], [4, 1]], 
                [[0, 2], [1, 2], [2, 2], [3, 2], [4, 2]]]
        ]

        *
        Then, we will extract every sub element's value into sub_element_value variable.
        First, for-loop to get every element of vertical_index. In this example, 
        we will respectively get the following values: 
        [[0, 0], [1, 0], [2, 0], [3, 0], [4, 0]]
        [[0, 1], [1, 1], [2, 1], [3, 1], [4, 1]]
        [[0, 2], [1, 2], [2, 2], [3, 2], [4, 2]]

        *
        In the first for-loop, we will get the value [1, 4, 0, 0, 4] initially.
        That means [0,0] = 1, [1,0] = 4, [2,0] = 0, [3,0] = 0, [4,0] = 4.

        then we will try to check if the value of the sub_element_value is equal to EMPTY(0).
        index_of_sub_element_zero = [i for i in range(len(sub_element_value)) if sub_element_value[i] == EMPTY]

        Here, we will get each index of the value which is equal to 0 from this sub_element_value list and
        save them into index_of_sub_element_zero.

        *
        To use the built-in function list.remove(0), we will remove the 0 from the sub_element_value list.
        sub_element_value.remove(EMPTY)

        Then add back the value of index_of_sub_element_zero to sub_element_value at the beginning of the list 
        to keep the same length of sub_element_value before removing the elements which value is 0.
        sub_element_value[:0] = [0] * len(index_of_sub_element_zero)

        * 
        At last, we will use the new values of sub_element_value to update the board.
        board[row][col] = sub_element_value[row]

        * 
        Then for-loop to keep processing the rest of the vertial_index.
    '''  
    _ = []     
    vertical_index = []
    sub_element_value = [] 

    # Get every element of vertical_index from board
    for col in range(COL):
        for row in range(ROW):
            _.append([row, col])
        vertical_index.append(_)
        _ = []
    
    # Extract every sub element's value of vertical_index into sub_element_value
    for _ in vertical_index:
        printd(f"{_}\n")
        for coord in _:   
            row, col = coord
            sub_element_value.append(board[row][col])

        printd(f"\tsub_element_value {sub_element_value}\n")

        index_of_sub_element_zero = [i for i in range(len(sub_element_value)) if sub_element_value[i] == EMPTY]
        
        printd(f"\tindex_of_sub_element_zero {index_of_sub_element_zero}\n")

        # Suggestion by ChatGPT, but seems no difference before and after reversing
        index_of_sub_element_zero.reverse()

        for i in index_of_sub_element_zero:
            sub_element_value.remove(EMPTY)

        printd(f"\tNew sub_element_value without 0 {sub_element_value}\n")


        # New values of sub_element_value
        sub_element_value[:0] = [0] * len(index_of_sub_element_zero)

        printd(f"\tNew sub_element_value with 0 adding {sub_element_value}\n\n\n")
        for coord in _:
            row, col = coord
            board[row][col] = sub_element_value[row]
        
        sub_element_value = []

    return board


# Find consecutive squares which have the same contents
# Then crush out them, drop them to fill the empty squares 
# And generate new candies on empty squares
# Finally, return the new board
def stableBoard(board, rounds):
    global current_score
    while True:
        consecutive_squares = getConsecutiveSquares(board)
        board, is_stable, score = crushOut(consecutive_squares, board)
        current_score = current_score + score
        if is_stable is True: # Stable board
            break
        else:
            board = moveDown(board)
            board = fillEmpty(board)
            displayBoard(board, rounds, current_score, True)
            print()
    return board



# Ask user for input about board size, how many rounds, how many types of candy
def gameSetup():
    global ROW, COL, CANDY_TYPES, ROUNDS
    while True:
        user_input_size = input("Enter board size: (format as: row col, for example: 10 5) Enter without input will use default size 10x5: ")
        if user_input_size == '':
            break
        if len(user_input_size.split()) == 2:
            if user_input_size.replace(' ', '').isdigit():
                if int(user_input_size.split()[0]) >=5 and int(user_input_size.split()[0]) <= 12 and int(user_input_size.split()[1]) >= 5 and int(user_input_size.split()[1]) <= 12:
                    break
                else:
                    print("Board size must be between 5 and 12. Please re-enter.")
            else:
                print("Please enter two integers separated by space. Please re-enter.")
        else:
            print("Please enter two integers separated by space. Please re-enter.")

    while True:
        user_input_rounds = input("Enter number of rounds, range is 3-15 (default: 10 if entered):")
        if user_input_rounds == '':
            break
        if user_input_rounds.isdigit():
            if int(user_input_rounds) >= 3 and int(user_input_rounds) <= 15:
                break
            else:
                print("Rounds must be between 3 and 15. Please re-enter.")
        else:
            print("Rounds must be integer. Please re-enter.")
        
    while True:
        user_input_candy = input("Enter number of types of candy, range is 4-8 (default: 5 if entered): ")
        if user_input_candy == '':
            break
        if user_input_candy.isdigit():
            if int(user_input_candy) >= 4 and int(user_input_candy) <= 8:
                break
            else:
                print("Candy types must be between 4 and 8. Please re-enter.")
        else:
            print("Candy types must be integer. Please re-enter.")
            
    if user_input_size:
        ROW = int(user_input_size.split(" ")[0])
        COL = int(user_input_size.split(" ")[1])
    
    if user_input_rounds:
        ROUNDS = int(user_input_rounds)
    
    if user_input_candy:
        CANDY_TYPES = int(user_input_candy)
    
    print()
    print(f"Game Setup ==> BOARD SIZE: {ROW}x{COL}, ROUNDS: {ROUNDS}, CANDY_TYPES: {CANDY_TYPES}. Ready to play!!")



# Fill the empty squares with random candies
def fillEmpty(board):

    ROW = len(board)
    COL = len(board[0])

    for row in range(ROW):
        for col in range(COL):
            if board[row][col] == EMPTY:
                board[row][col] = random.randint(1, CANDY_TYPES)
    return board



def isSwitchValid(row, col, direction):
    if direction == "u":
        if row == 0:
            return False
        else:
            return True
    elif direction == "d":
        if row == ROW - 1:
            return False
        else:
            return True
    elif direction == "l":
        if col == 0:
            return False
        else:
            return True
    elif direction == "r":
        if col == COL - 1:
            return False
        else:
            return True



def switchTwoCandies(board):
    while True:
        while True:            
            user_input = input("Example:3au or 4 b r: ").replace(" ", "").lower()

            if user_input.isalnum() and len(user_input) == 4 or len(user_input) == 3:   
                if len(user_input) == 4:
                    if user_input[:2].isdigit():
                        row = int(user_input[:2])
                        if (row >=0 and row <= ROW):
                            if user_input[2].isalpha():
                                col = ord(user_input[2]) - ord('a')
                                if (col >=0 and col <= COL):
                                    if user_input[3] in ["l", "r", "u", "d"]:
                                        direction = user_input[3]
                                        break
                                    else:   
                                        print("Invalid input: Direction is not valid! Please re-enter!")
                                else:
                                    print("Invalid input: Col is not within the range! Please re-enter!")
                            else:
                                print("Invalid input: Col is not alphabet! Please re-enter!")
                        else:
                            print("Invalid input: Row is not within the range! Please re-enter!")
                    else:
                        print("Invalid input: Row is not numeic! Please re-enter!")
                        
                
                if len(user_input) == 3:
                    if user_input[0].isdigit():
                        row = int(user_input[0])
                        if (row >=0 and row <= ROW):
                            if user_input[2].isalpha():
                                col = ord(user_input[1]) - ord('a')
                                if (col >=0 and col <= COL):
                                    if user_input[2] in ["l", "r", "u", "d"]:
                                        direction = user_input[2]
                                        break
                                    else:   
                                        print("Invalid input: Direction is not valid! Please re-enter!")
                                else:
                                    print("Invalid input: Col is not within the range! Please re-enter!")
                            else:
                                print("Invalid input: Col is not alphabet! Please re-enter!")
                        else:
                            print("Invalid input: Row is not within the range! Please re-enter!")
                    else:
                        print("Invalid input: Row is not numeic! Please re-enter!")
            else:    
                print("Invalidinput: Please re-enter!")
        
        is_switch_valid = isSwitchValid(row, col, direction)
        if is_switch_valid:
            switch_a_row = row
            switch_a_col = col

            if direction == "u":
                switch_b_row = row - 1
                switch_b_col = col
            elif direction == "d":
                switch_b_row = row + 1
                switch_b_col = col
            elif direction == "l":
                switch_b_row = row
                switch_b_col = col - 1
            elif direction == "r":
                switch_b_row = row
                switch_b_col = col + 1

        # create a new board
        switch_A = board[switch_a_row][switch_a_col]
        switch_B = board[switch_b_row][switch_b_col]

        board[switch_a_row][switch_a_col] = switch_B
        board[switch_b_row][switch_b_col] = switch_A

        # Check the new board if it is stable
        is_stable, _ = checkIsStable(board)

        if is_stable is True:
            # Return to original board
            board[switch_a_row][switch_a_col] = switch_A
            board[switch_b_row][switch_b_col] = switch_B

            print("Nothing Happened! Please re-neter!")
        else:
            break
    
    return board


rounds = 0
is_first = True
# gameSetup()
print("<< Welcome to Candy Crush Game Console version! >>")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
board = fillEmpty(createBoard())
displayBoard(board, is_score = True)
print()
while True:
    board = stableBoard(board, rounds)
    if rounds == ROUNDS:
        print("Game Over! Your score is: ", current_score)  
        break
    if is_first:
        print("Please enter one candy coordinate and direction(u/d/l/r) with or without space.")
        is_first = False
    board = switchTwoCandies(board)
    rounds += 1


<< Welcome to Candy Crush Game Console version! >>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

ROUND: 3 SCORE: 0
   +---+---+---+---+---+---+
 0 | 5 | 3 | 2 | 5 | 2 | 4 | 
   +---+---+---+---+---+---+
 1 | 1 | 2 | 5 | 4 | 3 | 5 | 
   +---+---+---+---+---+---+
 2 | 5 | 5 | 4 | 2 | 1 | 2 | 
   +---+---+---+---+---+---+
 3 | 4 | 1 | 4 | 2 | 1 | 2 | 
   +---+---+---+---+---+---+
 4 | 2 | 5 | 5 | 4 | 5 | 4 | 
   +---+---+---+---+---+---+
 5 | 5 | 2 | 5 | 3 | 1 | 4 | 
   +---+---+---+---+---+---+
 6 | 5 | 3 | 3 | 5 | 4 | 4 | 
   +---+---+---+---+---+---+
 7 | 2 | 4 | 1 | 3 | 3 | 3 | 
   +---+---+---+---+---+---+
 8 | 3 | 2 | 4 | 1 | 3 | 1 | 
   +---+---+---+---+---+---+
 9 | 4 | 2 | 1 | 2 | 2 | 3 | 
   +---+---+---+---+---+---+
     a   b   c   d   e   f   


Found the same candies more than 3 in-a-row!
Crushing out!
   +---+---+---+---+---+---+
 0 | 5 | 3 | 2 | 5 | 2 | 4 | 
   +---+---+---+---+---+---+
 1 | 1 | 2 | 5 | 4 | 3 | 5 | 
   +---+---+---+---+---+---+
 2 | 5 | 5 | 4 | 2 | 1

In [226]:
gameSetup()


Enter board size: (format as: row col, for example: 10 5) Enter without input will use default size 10x5: 
Enter number of rounds, range is 3-15 (default: 10 if entered):
Enter number of types of candy, range is 4-8 (default: 5 if entered): 

Game Setup ==> BOARD SIZE: 10x6, ROUNDS: 3, CANDY_TYPES: 5. Done!!


In [230]:
rounds = 0
is_first = True
# gameSetup()
board = fillEmpty(createBoard())
displayBoard(board, is_score = True)
print()
while True:
    board = stableBoard(board, rounds)
    if rounds == ROUNDS:
        print("Game Over! Your score is: ", current_score)  
        break
    if is_first:
        print("Please enter one candy coordinate and direction(u/d/l/r) with or without space.")
        is_first = False
    board = switchTwoCandies(board)
    rounds += 1



ROUND: 3 SCORE: 0
   +---+---+---+---+---+---+
 0 | 1 | 2 | 5 | 1 | 1 | 1 | 
   +---+---+---+---+---+---+
 1 | 4 | 2 | 5 | 4 | 2 | 4 | 
   +---+---+---+---+---+---+
 2 | 4 | 2 | 3 | 1 | 2 | 3 | 
   +---+---+---+---+---+---+
 3 | 5 | 3 | 5 | 1 | 4 | 5 | 
   +---+---+---+---+---+---+
 4 | 3 | 3 | 5 | 2 | 2 | 5 | 
   +---+---+---+---+---+---+
 5 | 5 | 4 | 4 | 2 | 3 | 3 | 
   +---+---+---+---+---+---+
 6 | 2 | 3 | 5 | 5 | 1 | 5 | 
   +---+---+---+---+---+---+
 7 | 3 | 5 | 4 | 5 | 1 | 1 | 
   +---+---+---+---+---+---+
 8 | 5 | 3 | 5 | 2 | 1 | 4 | 
   +---+---+---+---+---+---+
 9 | 2 | 2 | 2 | 1 | 1 | 4 | 
   +---+---+---+---+---+---+
     a   b   c   d   e   f   


Found the same candies more than 3 in-a-row!
Crushing out!
   +---+---+---+---+---+---+
 0 | 1 | 0 | 5 | 0 | 0 | 0 | 
   +---+---+---+---+---+---+
 1 | 4 | 0 | 5 | 4 | 2 | 4 | 
   +---+---+---+---+---+---+
 2 | 4 | 0 | 3 | 1 | 2 | 3 | 
   +---+---+---+---+---+---+
 3 | 5 | 3 | 5 | 1 | 4 | 5 | 
   +---+---+---+---+---+---+
 4 | 3